# Qubit 1-Norm
The objective is to find the 1-Norm of the qubit Hamiltonian. This Hamiltonian is defined as:
$$
\hat{\mathcal{H}}_{\rm qub} = \sum_i^{\mathcal{O}(N^4)} h_i \hat{P}_i
$$
which *could* be obtained from 
$$
\hat{\mathcal{H}} = \sum_{p q}^{N} h_{p q}^{} a_p^\dagger a_q^{}+ \frac{1}{2}\sum_{p q r s}^{N} h_{p q r s}^{} a_p^\dagger a_q^\dagger a_r^{} a_s^{}
$$
through a Jordan-Wigner transformation. This 1-Norm is then defined as the sum of absolute values of the coefficients:
$$
||\hat{\mathcal{H}}_{\rm qub}|| = \sum_i |h_i|
$$
Instead of doing this JW transformation first, this code gives you the option to directly compute the 1-Norm from any given molecular integrals $h_{pq}$ and $h_{pqrs}$ directly.

This formula reads:
$$
||\hat{\mathcal{H}}_{\rm qub}|| = |\tilde{h}| +  \sum_{p q}|\tilde{h}_{pq}| +\frac{1}{4}\sum_{\substack{p>q, r>s \\ p \neq q\neq r \neq s}}\left|h_{pqrs} - h_{pqsr}\right|\\
     + \frac{1}{8}\sum_{\substack{p q r \\ p \neq q \neq r}} \left|h_{pqrq} - 2h_{pqqr}\right| + \frac{1}{16}\sum_{\substack{p q \\ p \neq q}} \left|h_{pqpq}- 2h_{pqqp}\right|.
$$
if one starts from:
$$
\hat{\mathcal{H}} = \tilde{h}\mathbf{I} +\sum_{p,q} \tilde{h}_{pq} i \gamma_{2p}\gamma_{2q+1}
    + \frac{1}{2}\sum_{\substack{p q r s \\ p \neq q, r \neq s}} \tilde{h}_{pqrs} \gamma_{2p}\gamma_{2q}\gamma_{2r+1}\gamma_{2s+1}
$$
with
$$
\tilde{h} =\frac{1}{2} \sum_{p} h_{p p}+\frac{1}{8} \sum_{\substack{p q \\ p \neq q}}\left(h_{p q q p}-h_{p q p q}\right), \\
\tilde{h}_{p q} =\frac{1}{2} h_{p q}+\sum_{\substack{r \\ r \neq p, r\neq q}}\left(\frac{1}{4} h_{p r r q}-\frac{1}{8} h_{p q r r}\right), \\
\tilde{h}_{p q r s} =-\frac{1}{8}\left[1+\left(1-\delta_{p r}\right)\left(1-\delta_{q s}\right)\right] h_{p q r s}.
$$

See the overleaf for a proof.

In [1]:
import numpy as np
from openfermion.ops import FermionOperator, QubitOperator, InteractionOperator
from openfermion.transforms import jordan_wigner

In [2]:
# Define the above function, mapping an OpenFermion InteractionOperator to the qubit 1-Norm:
def JW1norm(molecular_hamiltonian):
    '''
    Returns the 1-Norm of the Hamiltonian after a Jordan-Wigner
    transformation.

    Parameters
    ----------
    molecular_hamiltonian : InteractionOperator object representing the molecular Hamiltonian

    Returns
    -------
    q1norm : 1-Norm of the Qubit Hamiltonian  
    '''
    
    constant = molecular_hamiltonian.constant
    one_body_coefficients = molecular_hamiltonian.one_body_tensor
    two_body_coefficients = 2*np.copy(molecular_hamiltonian.two_body_tensor)
    # There is a factor of 2 difference because of the convention to put
    # a factor 1/2 in front of the two-body sum in the Hamiltonian.
    

    
    htilde = constant # The constant here is just the nuclear repulsion 
                      # (+ the core adjustment in the case of an active space)
    for p in range(n_qubits):
        htilde += 1/2. * one_body_coefficients[p,p]
        for q in range(n_qubits):
            if q != p:
                htilde +=  1/8 * (two_body_coefficients[p,q,q,p] - two_body_coefficients[p,q,p,q])
    
    htildepq = np.zeros(one_body_coefficients.shape)
    for p in range(n_qubits):
        for q in range(n_qubits):
            htildepq[p,q] = 1/2 * one_body_coefficients[p,q]
            for r in range(n_qubits):
                if r!=p and r!=q :
                    htildepq[p,q] += ((1/4 * two_body_coefficients[p,r,r,q]) - \
                                      (1/8 * two_body_coefficients[p,q,r,r]))
    
    q1norm = abs(htilde) + np.sum(np.absolute(htildepq)) 
    for p in range(n_qubits):
        for q in range(n_qubits):
            if p != q:
                q1norm += 1/16 * abs(two_body_coefficients[p,q,p,q]-2*two_body_coefficients[p,q,q,p])
            for r in range(n_qubits):
                if p != q and q!= r and p!=r:
                    q1norm += 1/8 * abs(two_body_coefficients[p,q,r,q] - \
                                        2 * two_body_coefficients[p,q,q,r])
                for s in range(n_qubits):
                    if p>q and r>s and p!=q and p!=r and p!=s and q!=r and q!=s and r!=s:
                        q1norm += 1/4 * abs(two_body_coefficients[p,q,r,s] - \
                                            two_body_coefficients[p,q,s,r])
    return q1norm

First, we can try different artificial sets of integrals.

In [3]:
nmo = 4
n_qubits = 2 * nmo
np.random.seed(1234) # You can leave this out if you want
c2b = np.zeros((n_qubits,n_qubits,n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(n_qubits):
        for k in range(n_qubits):
            for l in range(n_qubits):
                c2b[i,j,k,l]=100*np.random.rand(1)-30
                c2b[j,i,l,k]=c2b[i,j,k,l]
                c2b[l,k,j,i]=c2b[i,j,k,l]
                c2b[k,l,i,j]=c2b[i,j,k,l]
                c2b[i,k,j,l]=c2b[i,j,k,l]
                c2b[l,j,k,i]=c2b[i,j,k,l]
                c2b[k,i,l,j]=c2b[i,j,k,l]
                c2b[j,l,i,k]=c2b[i,j,k,l]

c1b = np.zeros((n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(i+1):
        c1b[i,j]=100*np.random.rand(1)-30
        c1b[j,i] = c1b[i,j]

In [4]:
# "Confirm" it has the right symmetries...
c2b[0,1,2,3] == c2b[0,2,1,3] == c2b[3,1,2,0] == c2b[3,2,1,0]==\
c2b[1,0,3,2] == c2b[1,3,0,2] == c2b[2,3,0,1] == c2b[2,3,0,1]

True

In [5]:
constant = 0
molecular_hamiltonian = InteractionOperator(constant, c1b, c2b)
q1norm = JW1norm(molecular_hamiltonian)

In [6]:
print("Qubit 1-norm calculated with the formula:",q1norm,
      "\nReal qubit 1-norm is:", (jordan_wigner(molecular_hamiltonian)).induced_norm())

Qubit 1-norm calculated with the formula: 15657.16913658239 
Real qubit 1-norm is: 15624.30914495515


As you can see, it is slightly off. We can do the same, but for a Hamiltonian with only one-body terms and off-diagonal two-body terms (and a constant if you want):

In [7]:
c2b = np.zeros((n_qubits,n_qubits,n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(n_qubits):
        for k in range(n_qubits):
            for l in range(n_qubits):
                if i != j and i != k and i != l and j != k and j != l and k != l:
                    c2b[i,j,k,l]=100*np.random.rand(1)-30
                    c2b[j,i,l,k]=c2b[i,j,k,l]
                    c2b[l,k,j,i]=c2b[i,j,k,l]
                    c2b[k,l,i,j]=c2b[i,j,k,l]
                    c2b[i,k,j,l]=c2b[i,j,k,l]
                    c2b[l,j,k,i]=c2b[i,j,k,l]
                    c2b[k,i,l,j]=c2b[i,j,k,l]
                    c2b[j,l,i,k]=c2b[i,j,k,l]

c1b = np.zeros((n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(i+1):
        c1b[i,j]=100*np.random.rand(1)-30
        c1b[j,i] = c1b[i,j]

constant = -50
molecular_hamiltonian_2 = InteractionOperator(constant,c1b,c2b)

In [8]:
print("Qubit 1-norm calculated with the formula:",JW1norm(molecular_hamiltonian_2),
      "\nReal qubit 1-norm is:", (jordan_wigner(molecular_hamiltonian_2)).induced_norm())

Qubit 1-norm calculated with the formula: 8102.886761467898 
Real qubit 1-norm is: 8102.886761467895


And now it does work. This means the mistake lies somewhere in the not total off-diagonal terms of the 2bdy integrals. 

The old method I had for calculating the qubit 1-norm is the following function, which first needed to normal order the integrals. It is quite involved and probably not so readable. It has three parts, the first sums go over all the terms with four X or Y pauli strings, the second part over terms with two X or Y Paulis and the last part over terms with no X or Y pauli strings. The formula for this norm is given by:
$$
||H_\text{qub}|| =  \sum_{p>q>r>s}\frac{1}{4}\Bigg( \text{max}\left(|\tilde{h}_{pqrs} + \tilde{h}_{prqs} + \tilde{h}_{psqr}|, |\tilde{h}_{qrps} + \tilde{h}_{qspr} + \tilde{h}_{rspq}|\right) \\
     +  \text{max}\left(|-\tilde{h}_{pqrs} + \tilde{h}_{prqs} + \tilde{h}_{psqr}|, |\tilde{h}_{qrps} + \tilde{h}_{qspr} - \tilde{h}_{rspq}|\right) \\
     +  \text{max}\left(|\tilde{h}_{pqrs} - \tilde{h}_{prqs} + \tilde{h}_{psqr}|, |\tilde{h}_{qrps} - \tilde{h}_{qspr} + \tilde{h}_{rspq}|\right) \\
     +  \text{max}\left(|\tilde{h}_{pqrs} + \tilde{h}_{prqs} - \tilde{h}_{psqr}|, |-\tilde{h}_{qrps} + \tilde{h}_{qspr} + \tilde{h}_{rspq}|\right) \Bigg)\\
    + \sum_{p>q} \frac{1}{2} \Bigg(  \sum_{r < q} \text{max}\left(|\tilde{h}_{prqr}|, |\tilde{h}_{qrpr}|\right) + \sum_{r>q, r<p} \text{max}\left(|\tilde{h}_{prrq}|, |\tilde{h}_{rqpr}|\right)\\
+ \sum_{r>p} \text{max}\left( |\tilde{h}_{rprq}|, |\tilde{h}_{rqrp}| \right)  \\
+ \quad \, \, \, \text{max}\big(\big|-2 h_{pq} + \sum_{r<q} \tilde{h}_{prqr} - \sum_{r>q, r<p} \tilde{h}_{prrq} + \sum_{r>p} \tilde{h}_{rprq}\big|, \\
 \quad\quad \, \,  \big| -2 h_{qp} + \sum_{r<q} \tilde{h}_{qrpr} - \sum_{r>q, r<p} \tilde{h}_{rqpr} + \sum_{r>p} \tilde{h}_{rqrp}\big|\big) \Bigg)\\
    + \frac{1}{4}\Bigg(\sum_{p>q}  |\tilde{h}_{pqpq}| + \sum_{p}\big|-2 h_{pp} + \sum_{q<p} \tilde{h}_{pqpq} + \sum_{q>p} \tilde{h}_{qpqp}\big| \\
    + \big|-2 \sum_{p} h_{pp} + \sum_{p>q} \tilde{h}_{pqpq}\big| \Bigg).
$$
where it uses "normal ordered" integrals:
$$
\tilde{h}_{pqrs}= \left(h_{pqrs} - h_{pqsr} - h_{qprs} + h_{qpsr}\right),
$$
It assumes no symmetry whatsoever of the integrals, and could be simplified significantly if one were to assume symmetries of the integrals (all the max functions can be eliminated). I'm looking into doing that and comparing it with the new formula....

In [9]:
def normal_order_tbc(two_body_coefficients, n_qubits):
    '''
    Normal order the Hamiltonian giving the right two-body coefficients

    Parameters
    ----------
    two_body_coefficients : A numpy array of size
        (n_qubits, n_qubits, n_qubits, n_qubits)
    n_qubits : Number of qubits

    Returns
    -------
    two_body_coefficients : A numpy array of size
        (n_qubits, n_qubits, n_qubits, n_qubits)
    '''
    print("Normal ordering.....")
    
    for i in range(n_qubits):
        for j in range(n_qubits):
            for k in range(n_qubits):
                for l in range(n_qubits):
                    if i == j or k == l:
                        two_body_coefficients[i,j,k,l] = 0.
                    elif i > j and k > l:
                        two_body_coefficients[i,j,k,l] = (
                        two_body_coefficients[i,j,k,l] -
                        two_body_coefficients[i,j,l,k] -
                        two_body_coefficients[j,i,k,l] +
                        two_body_coefficients[j,i,l,k])
                        
                        two_body_coefficients[i,j,l,k] = 0.
                        two_body_coefficients[j,i,k,l] = 0.
                        two_body_coefficients[j,i,l,k] = 0.
    print("Done normal ordering")
    return two_body_coefficients

def JW1norm_old(molecular_hamiltonian, normal_order=True):
    '''
    Returns the 1-Norm of the Hamiltonian after a Jordan-Wigner
    transformation given normal ordered one-body (2D np.array)
    and two-body (4D np.array) coefficients.

    Parameters
    ----------
    molecular_hamiltonian : InteractionOperator object representing the molecular Hamiltonian
    normal_order : Boolean, optional
        Whether to normal order the Hamiltonian (If false, assumes that
        the Hamiltonian is already in normal ordered form). The default is True.

    Returns
    -------
    q1norm : 1-Norm of the Qubit Hamiltonian  
    '''
    
    constant = molecular_hamiltonian.constant
    one_body_coefficients = molecular_hamiltonian.one_body_tensor
    two_body_coefficients = np.copy(molecular_hamiltonian.two_body_tensor)
    
    n_qubits = one_body_coefficients.shape[0]
    if normal_order:
        two_body_coefficients = normal_order_tbc(two_body_coefficients, n_qubits)
        
    q1norm = 0           

      
    # p != r != q != s
    for i in range(n_qubits):
        for j in range(i):
            for k in range(j):
                for l in range(k):
                    ijkl = two_body_coefficients[i,j,k,l]
                    ikjl = two_body_coefficients[i,k,j,l]
                    iljk = two_body_coefficients[i,l,j,k]
                    jkil = two_body_coefficients[j,k,i,l]
                    jlik = two_body_coefficients[j,l,i,k]
                    klij = two_body_coefficients[k,l,i,j]
                    q1norm += 1/4 * max(abs(ijkl + ikjl + iljk),
                                        abs(jkil + jlik + klij))
                    q1norm += 1/4 * max(abs(-ijkl + ikjl + iljk),
                                        abs(jkil + jlik - klij))
                    q1norm += 1/4 * max(abs(ijkl - ikjl + iljk),
                                        abs(jkil - jlik + klij))
                    q1norm += 1/4 * max(abs(ijkl + ikjl - iljk),
                                        abs(-jkil + jlik + klij))
    
    # p = r or q = s
    for i in range(n_qubits):
        for j in range(i):
            temp_a = - 2 * one_body_coefficients[i,j]
            temp_b = - 2 * one_body_coefficients[j,i]
            for k in range(j):
                temp_a += two_body_coefficients[i,k,j,k]
                temp_b += two_body_coefficients[j,k,i,k]
                q1norm += 1/2 * max(abs(two_body_coefficients[i,k,j,k]),\
                                    abs(two_body_coefficients[j,k,i,k]))
                
            for k in range(j+1,i):
                temp_a -= two_body_coefficients[i,k,k,j]
                temp_b -= two_body_coefficients[k,j,i,k]
                q1norm += 1/2 * max(abs(two_body_coefficients[i,k,k,j]),\
                                    abs(two_body_coefficients[k,j,i,k]))

            for k in range(i+1,n_qubits):
                temp_a += two_body_coefficients[k,i,k,j]
                temp_b += two_body_coefficients[k,j,k,i]
                q1norm += 1/2 * max(abs(two_body_coefficients[k,i,k,j]),\
                                    abs(two_body_coefficients[k,j,k,i]))

            q1norm += 1/2 * max(abs(temp_a), abs(temp_b))
    
    # p = r, q = s
    cs = [0 for i in range((n_qubits+2))]
    cs[-1] = -4*constant
    for i in range(n_qubits):
        cs[-1] -= 2* one_body_coefficients[i,i]
        for j in range(i):                                         
            cs[0] += abs(two_body_coefficients[i,j,i,j])
            cs[-1] += two_body_coefficients[i,j,i,j]
    for i in range(n_qubits):
        cs[i+1] -= 2 * one_body_coefficients[i,i]
        for j in range(n_qubits):
            if j > i:
                cs[i+1] += two_body_coefficients[j,i,j,i]
                
            elif j < i:
                cs[i+1] += two_body_coefficients[i,j,i,j]
    q1norm += 1/4 * sum(list(map(abs,cs)))
    return q1norm

In [10]:
c2b = np.zeros((n_qubits,n_qubits,n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(n_qubits):
        for k in range(n_qubits):
            for l in range(n_qubits):
                c2b[i,j,k,l]=100*np.random.rand(1)-30
                c2b[j,i,l,k]=c2b[i,j,k,l]
                c2b[l,k,j,i]=c2b[i,j,k,l]
                c2b[k,l,i,j]=c2b[i,j,k,l]
                c2b[i,k,j,l]=c2b[i,j,k,l]
                c2b[l,j,k,i]=c2b[i,j,k,l]
                c2b[k,i,l,j]=c2b[i,j,k,l]
                c2b[j,l,i,k]=c2b[i,j,k,l]

c1b = np.zeros((n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(i+1):
        c1b[i,j]=100*np.random.rand(1)-30
        c1b[j,i] = c1b[i,j]     

constant = 0

molecular_hamiltonian_3 = InteractionOperator(constant,c1b,c2b)

In [11]:
print("Qubit 1-norm calculated with the old formula:",JW1norm_old(molecular_hamiltonian_3),
      "\nReal qubit 1-norm is:", (jordan_wigner(molecular_hamiltonian_3)).induced_norm())

Normal ordering.....
Done normal ordering
Qubit 1-norm calculated with the old formula: 16520.667199007126 
Real qubit 1-norm is: 16520.66719900718


Turns out the real formula can be written as:
$$
||\hat{\mathcal{H}}_{\rm qub}|| = |\tilde{h}| +  \sum_{p q}|\tilde{h}_{pq}| +\frac{1}{4}\sum_{\substack{p>q, r>s \\ p \neq q\neq r \neq s}}\left|h_{pqrs} - h_{pqsr}\right|\\
     + \frac{1}{4}\sum_{\substack{p q r \\ p \neq q \neq r}} \left|h_{pqrq} - h_{pqqr}\right| + \frac{1}{8}\sum_{\substack{p q \\ p \neq q}} \left|h_{pqpq}- h_{pqqp}\right|.
$$
with
$$
\tilde{h} =\frac{1}{2} \sum_{p} h_{p p}+\frac{1}{8} \sum_{\substack{p q \\ p \neq q}}\left(h_{p q q p}-h_{p q p q}\right), \\
\tilde{h}_{p q} =\frac{1}{2} h_{p q}+\sum_{\substack{r \\ r \neq p, r\neq q}}\frac{1}{4}\left( h_{p r r q}- h_{p q r r}\right), \\
\tilde{h}_{pqrs} = \frac{1}{4} h_{pqrs}
$$
indicating that the Hamiltonian in majorana representation is actually written with these coefficients:
$$
\hat{\mathcal{H}} = \tilde{h}\mathbf{I} +\sum_{p,q} \tilde{h}_{pq} i \gamma_{2p}\gamma_{2q+1}
    + \frac{1}{2}\sum_{\substack{p q r s \\ p \neq q, r \neq s}} \tilde{h}_{pqrs} \gamma_{2p}\gamma_{2q}\gamma_{2r+1}\gamma_{2s+1}
$$

In [15]:
# Define the new function, mapping an OpenFermion InteractionOperator to the qubit 1-Norm:
def JW1norm_new(molecular_hamiltonian):
    '''
    Returns the 1-Norm of the Hamiltonian after a Jordan-Wigner
    transformation.

    Parameters
    ----------
    molecular_hamiltonian : InteractionOperator object representing the molecular Hamiltonian

    Returns
    -------
    q1norm : 1-Norm of the Qubit Hamiltonian  
    '''
    
    constant = molecular_hamiltonian.constant
    one_body_coefficients = molecular_hamiltonian.one_body_tensor
    two_body_coefficients = 2*np.copy(molecular_hamiltonian.two_body_tensor)
    # There is a factor of 2 difference because of the convention to put
    # a factor 1/2 in front of the two-body sum in the Hamiltonian.
    

    
    htilde = constant # The constant here is just the nuclear repulsion 
                      # (+ the core adjustment in the case of an active space)
    for p in range(n_qubits):
        htilde += 1/2. * one_body_coefficients[p,p]
        for q in range(n_qubits):
            if q != p:
                htilde +=  1/8 * (two_body_coefficients[p,q,q,p] - two_body_coefficients[p,q,p,q])
    
    htildepq = np.zeros(one_body_coefficients.shape)
    for p in range(n_qubits):
        for q in range(n_qubits):
            htildepq[p,q] = 1/2 * one_body_coefficients[p,q]
            for r in range(n_qubits):
                if r!=p and r!=q :
                    htildepq[p,q] += ((1/4 * two_body_coefficients[p,r,r,q]) - \
                                      (1/4 * two_body_coefficients[p,r,q,r]))
    
    q1norm = abs(htilde) + np.sum(np.absolute(htildepq)) 
    for p in range(n_qubits):
        for q in range(n_qubits):
            if p != q:
                q1norm += 1/8 * abs(two_body_coefficients[p,q,p,q]-two_body_coefficients[p,q,q,p])
            for r in range(n_qubits):
                if p != q and q!= r and p!=r:
                    q1norm += 1/4 * abs(two_body_coefficients[p,q,r,q] - \
                                        two_body_coefficients[p,q,q,r])
                for s in range(n_qubits):
                    if p>q and r>s and p!=q and p!=r and p!=s and q!=r and q!=s and r!=s:
                        q1norm += 1/4 * abs(two_body_coefficients[p,q,r,s] - \
                                            two_body_coefficients[p,q,s,r])
    return q1norm

In [24]:
nmo = 4
n_qubits = 2 * nmo
# np.random.seed(1234) # You can leave this out if you want
c2b = np.zeros((n_qubits,n_qubits,n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(n_qubits):
        for k in range(n_qubits):
            for l in range(n_qubits):
                c2b[i,j,k,l]=100*np.random.rand(1)-30
                c2b[j,i,l,k]=c2b[i,j,k,l]
                c2b[l,k,j,i]=c2b[i,j,k,l]
                c2b[k,l,i,j]=c2b[i,j,k,l]
                c2b[i,k,j,l]=c2b[i,j,k,l]
                c2b[l,j,k,i]=c2b[i,j,k,l]
                c2b[k,i,l,j]=c2b[i,j,k,l]
                c2b[j,l,i,k]=c2b[i,j,k,l]

c1b = np.zeros((n_qubits,n_qubits))
for i in range(n_qubits):
    for j in range(i+1):
        c1b[i,j]=100*np.random.rand(1)-30
        c1b[j,i] = c1b[i,j]
        
constant = -50
molecular_hamiltonian_4 = InteractionOperator(constant,c1b,c2b)

In [25]:
print("Qubit 1-norm calculated with the new formula:",JW1norm_new(molecular_hamiltonian_4),
      "\nReal qubit 1-norm is:", (jordan_wigner(molecular_hamiltonian_4)).induced_norm())

Qubit 1-norm calculated with the new formula: 16517.541487975508 
Real qubit 1-norm is: 16517.5414879755
